In [1]:
import org.apache.spark.sql.functions._
import org.apache.spark.sql.expressions._
import scala.util.{Failure, Success, Try}
import org.apache.spark.sql.types._
import org.apache.spark.sql.SparkSession

import java.time.LocalDateTime
import java.time.LocalDate
import java.time.format.DateTimeFormatter
import java.time.temporal.ChronoUnit.DAYS

VBox()

Starting Spark application


The code failed because of a fatal error:
	Session 50 unexpectedly reached final status 'dead'. See logs:
stdout: 

stderr: 
24/03/15 04:30:41 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/03/15 04:30:41 INFO DefaultNoHARMFailoverProxyProvider: Connecting to ResourceManager at ip-10-0-15-66.ap-northeast-2.compute.internal/10.0.15.66:8032
24/03/15 04:30:42 INFO Configuration: resource-types.xml not found
24/03/15 04:30:42 INFO ResourceUtils: Unable to find 'resource-types.xml'.
24/03/15 04:30:42 INFO Client: Verifying our application has not requested more than the maximum memory capability of the cluster (12288 MB per container)
24/03/15 04:30:42 INFO Client: Will allocate AM container, with 2432 MB memory including 384 MB overhead
24/03/15 04:30:42 INFO Client: Setting up container launch context for our AM
24/03/15 04:30:42 INFO Client: Setting up the launch environment for our AM container
24/03/15 04:

In [ ]:
def dateTimeFormatter = DateTimeFormatter.ofPattern("yyyyMMddHHmmss")
def appName: String = LocalDateTime.now().format(dateTimeFormatter)
def previousDay(minusDays: Long) = {
    LocalDateTime.now().minusDays(minusDays).format(dateTimeFormatter).substring(0, 8)
}

In [ ]:
val appName = LocalDateTime.now().format(DateTimeFormatter.ofPattern("yyyyMMddHHmmss"))

val spark = (
    SparkSession
    .builder()
    .appName(appName)
    .config("spark.cores.max", 8)
    .config("spark.executor.cores", 8)
    .config("spark.executor.memory", "36g")
    .getOrCreate()
    )

In [ ]:
val mongoUrl = "mongodb+srv://xxxxxxxxxxxxxxxxxxxxxx/"

In [ ]:
val fsDf = (
    spark.read.format("mongodb")
    .option("spark.mongodb.read.connection.uri", mongoUrl)
    .option("spark.mongodb.write.connection.uri", mongoUrl)
    .option("database", "coreEngine")
    .option("collection", "ReportRiskPremiumCalcurate")
    .option("aggregation.pipeline", """
[
  {
    $group: {
      _id: "$stockCode",
      stock: {
        $top: {
          output: "$$ROOT",
          sortBy: { updateDate: -1 },
        },
      },
    },
  },
  { $replaceRoot: { newRoot: "$stock" } },
]
""")
    .load()
    .drop("_id", "updateDate")
    .withColumnRenamed("riskPremium", "FSRiskPremium")
    
    )

In [ ]:
val nsDf = (
    spark.read.format("mongodb")
    .option("spark.mongodb.read.connection.uri", mongoUrl)
    .option("spark.mongodb.write.connection.uri", mongoUrl)
    .option("database", "coreEngine")
    .option("collection", "NewsRiskPremium")
    .load()
    .drop("_id", "riskParagraph", "wholeParagraph")
    .withColumnRenamed("riskPremium", "NSRiskPremium")
    )

In [ ]:
val hfDf = (
    spark.read.format("mongodb")
    .option("spark.mongodb.read.connection.uri", mongoUrl)
    .option("spark.mongodb.write.connection.uri", mongoUrl)
    .option("database", "coreEngine")
    .option("collection", "PriceRiskPremium")
    .load()
    .drop("_id", "stockFullName")
    .withColumnRenamed("riskPremium", "HFRiskPremium")
    )

In [ ]:
val matchPipe = String.format("{ $match: { updateDate: { $gte: '%s' } } }", previousDay(1555))
val indexDf = (
    spark.read.format("mongodb")
    .option("spark.mongodb.read.connection.uri", mongoUrl)
    .option("spark.mongodb.write.connection.uri", mongoUrl)
    .option("database", "coreEngine")
    .option("collection", "IndexComposition")
    .option("aggregation.pipeline", matchPipe)
    .load()
    .select("updateDate", "rank", "index", "isuSrtCd")
    .withColumnRenamed("isuSrtCd", "stockCode")
    )

In [ ]:
val kosdaq150Df = indexDf.where(col("index") === "kosdaq_150").withColumnRenamed("rank", "kosdaq150").drop("rank").drop("index")
val kospi200Df = indexDf.where(col("index") === "kospi_200").withColumnRenamed("rank", "kospi200").drop("rank").drop("index")
val krx100Df = indexDf.where(col("index") === "krx_100").withColumnRenamed("rank", "krx100").drop("rank").drop("index")
val krx300Df = indexDf.where(col("index") === "krx_300").withColumnRenamed("rank", "krx300").drop("rank").drop("index")

In [ ]:
val riskPremiumDf = (
    hfDf
    .join(fsDf, Seq("stockCode"), "left")
    .join(nsDf, Seq("stockCode", "updateDate"), "left")
    .na.fill(0)
    .withColumn("BDRiskPremium", lit(0))
    .withColumn("FNRiskPremium", lit(0))
    .withColumn("HFPctRank", percent_rank().over(Window.partitionBy("updateDate").orderBy("HFRiskPremium")))
    .withColumn("BDPctRank", percent_rank().over(Window.partitionBy("updateDate").orderBy("BDRiskPremium")))
    .withColumn("FNPctRank", percent_rank().over(Window.partitionBy("updateDate").orderBy("FNRiskPremium")))
    .withColumn("NSPctRank", percent_rank().over(Window.partitionBy("updateDate").orderBy("NSRiskPremium")))
    .withColumn("riskPremium", col("FSRiskPremium") + (col("HFRiskPremium") * lit(0.1)) + (col("BDRiskPremium") * lit(0.1)) + (col("FNRiskPremium") * lit(0.1)) + (col("NSRiskPremium") * lit(0.1)))
    .withColumn("prevRiskPremium", lag(col("riskPremium"), offset=-1).over(Window.partitionBy("stockCode").orderBy(["updateDate"])) - col("riskPremium"))
    .where(col("stockName").isNotNull)
    .withColumn("score", ((lit(1) - col("FSPctRank")) * 10000).cast(IntegerType))
    .withColumn("FSScore", ((lit(1) - col("FSPctRank")) * 10000).cast(IntegerType))
    .withColumn("HFScore", ((lit(1) - col("HFPctRank")) * 10000).cast(IntegerType))
    .withColumn("BDScore", ((lit(1) - col("BDPctRank")) * 10000).cast(IntegerType))
    .withColumn("FNScore", ((lit(1) - col("FNPctRank")) * 10000).cast(IntegerType))
    .withColumn("NSScore", ((lit(1) - col("NSPctRank")) * 10000).cast(IntegerType))
    .withColumn("ReRank", row_number().over(Window.partitionBy(col("updateDate")).orderBy(col("grade"), col("FSPctRank"))))
    .join(kosdaq150Df, Seq("stockCode", "updateDate"), "left")
    .join(kospi200Df, Seq("stockCode", "updateDate"), "left")
    .join(krx100Df, Seq("stockCode", "updateDate"), "left")
    .join(krx300Df, Seq("stockCode", "updateDate"), "left")
    .withColumn("fs", struct(col("FSRiskPremium").as("riskPremium"), col("FSScore").as("score")))
    .withColumn("hf", struct(col("HFRiskPremium").as("riskPremium"), col("HFScore").as("score")))
    .withColumn("bd", struct(col("BDRiskPremium").as("riskPremium"), col("BDScore").as("score")))
    .withColumn("fn", struct(col("FNRiskPremium").as("riskPremium"), col("FNScore").as("score")))
    .withColumn("ns", struct(col("NSRiskPremium").as("riskPremium"), col("NSScore").as("score")))
    .withColumn("detail", struct(col("fs"), col("hf"), col("bd"), col("fn"), col("ns")))
    .withColumn("rank", struct(col("ReRank"), col("kosdaq150"), col("kospi200"), col("krx100"), col("krx300")))
    .select("corpCls", "stockCode", "updateDate", "bsnsYear", "quarter", "grade", "stockName", "riskPremium", "score", "detail", "rank", "loanAvailable", "basicReturn", "expectedProfit", "expectedRisk", "profitLoss")

In [ ]:
"""
(
    riskPremiumDf
    .write.format("mongodb")
    .mode("append")
    .option("upsertDocument", "true")
    .option("idFieldList", "updateDate,stockCode")
    .option("spark.mongodb.read.connection.uri", mongoUrl)
    .option("spark.mongodb.write.connection.uri", mongoUrl)
    .option("database", "coreEngine")
    .option("collection", "RiskPremium")
    .save()
    )
"""